In [1]:
import numpy as np
import qutip as qt
import cmath

In [2]:
# Swap function
def swapPositions(list, pos1, pos2):
     
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list
 
# Driver function
#List = [23, 65, 19, 90]
pos1, pos2  = 2, 4
List = [0,1,2,3,4]
#print(swapPositions(List, pos1-1, pos2-1))

### BS Function

In [3]:
def BS(J, N, state, number_modes, mode1, mode2):
    #N = 2  # number of levels in qubit state
    #E = 1   # number of photons in the mode
    alpha = 1 # eigenvalue of coherent state
    t = 10  # evolution time
    steps = 1000 #time steps
    #J = 0.07853957008534297  # strength of the COUPLİNG for 50/50
    #kappa = 0  # strength of the Cross-Kerr interaction
    
    
    a = qt.qeye(N)
    b = qt.tensor(qt.destroy(N), qt.qeye(N))
    c = qt.tensor(qt.qeye(N), qt.destroy(N))
    
    H_TDC = J * (b.dag()*c + c.dag()*b)   # TDC, on one mode a new photon is created, on the other one is destroyed.
    tlist = np.linspace(0, t, steps)
    BS_all = qt.propagator(H_TDC, tlist, [], {}) 
    BS = BS_all[-1]
    
    #Creating a list [0,1,2,3,4] for the modes
    initial_permute_list = list(range(number_modes))  #[0,1,2,...]
    
    #Swapping the modes
    
    pos1, pos2  = 1,mode1+1
    
    #swapPositions is already defined above. 
    permuted_list = swapPositions(initial_permute_list, pos1-1, pos2-1)

    pos3, pos4 = 2,mode2+1
    permuted_list2 = swapPositions(permuted_list, pos3-1, pos4-1)

    
    state_perm = state.permute(permuted_list2)
    
    operator_list = [BS] + [a] * (number_modes - 2)

    q1 = qt.tensor(operator_list) * state_perm
    
    q2 = q1.permute(permuted_list)
    
    
    results = []
    results.append(q2)
    for i in range(number_modes):
        pt = q2.ptrace(i)
        
        results.append(np.trace(pt * qt.create(N) * qt.destroy(N)))
        
        
    return results

### PS

In [4]:
def PS(N, phi, state, number_modes, mode1):
    #N = 10  # number of levels in qubit state
    t = 10  # evolution time
    steps = 1000 #time steps
    
    a = qt.qeye(N)
    
    
    #phi = 0.1570909090909091
    x = [np.exp(1j*n*phi) for n in range(N)]
    PS = qt.Qobj(np.diag(x))
    
    #Creating a list [0,1,2,3,4] for the modes
    initial_permute_list = list(range(number_modes))  #[0,1,2,...]
    
    #Swapping the modes
    
    pos1, pos2  = 1,mode1+1
    
    #swapPositions is already defined above. 
    permuted_list = swapPositions(initial_permute_list, pos1-1, pos2-1)
    
    state_perm = state.permute(permuted_list)
    
    operator_list = [PS] + [a] * (number_modes - 1)

    q1 = qt.tensor(operator_list) * state_perm
    
    q2 = q1.permute(permuted_list)
    
    
    results = []
    results.append(q2)
    for i in range(number_modes):
        pt = q2.ptrace(i)
        results.append(np.trace(pt * qt.create(N) * qt.destroy(N)))
        
        
    return results

### CKH

In [5]:
def CKH2(N, kappa, state, number_modes, mode1, mode2,*args): #args := the modes we want to put into the partial trace
    #N = 10
    #alpha = 1
    t = 10  # evolution time
    steps = 1000 #time steps
    #J = 0.07853957008534297  # strength of the COUPLİNG for 50/50
    #kappa = 0  # strength of the Cross-Kerr interaction

    a = qt.qeye(N)
    a_V = qt.tensor(qt.destroy(N), qt.qeye(N))
    c = qt.tensor(qt.qeye(N), qt.destroy(N))
    
    H_int = kappa * a_V.dag() * a_V  * c.dag() * c
    
    tlist = np.linspace(0, t, steps)
    U_all = qt.propagator(H_int, tlist, [], {}) 
    U = U_all[-1]

    #Creating a list [0,1,2,3,4] for the modes
    initial_permute_list = list(range(number_modes))
    
    #Swapping the modes
    
    pos1, pos2  = 1, mode1+1
    
    #swapPositions is already defined above. 
    permuted_list = swapPositions(initial_permute_list, pos1-1, pos2-1)

    pos3, pos4 = 2,mode2+1
    permuted_list2 = swapPositions(permuted_list, pos3-1, pos4-1)
    state_perm = state.permute(permuted_list2)
    
    operator_list = [U] + [a] * (number_modes - 2)

    q1 = qt.tensor(operator_list) * state_perm
    
    q2 = q1.permute(permuted_list)
    
    
    results0 = []
    results0.append(q2)
    
    pt_mode_list = [] 
    for i in args:
        pt_mode_list.append(i)
        
    pt = q2.ptrace(pt_mode_list)
    results0.append(pt)
    alpha = (pt[1][0][N])   #|0E> 

    phi = np.angle(alpha)
     

    results0.append(phi)    
    
        
        
    return results0

## Identity

In [6]:
def identity(N, state, number_modes, mode1):
    
    a = qt.qeye(N)
    

    #Creating a list [0,1,2,3,4] for the modes
    initial_permute_list = list(range(number_modes))  #[0,1,2,...]
    
    #Swapping the modes
    
    pos1, pos2  = 1,mode1+1
    
    #swapPositions is already defined above. 
    permuted_list = swapPositions(initial_permute_list, pos1-1, pos2-1)
    
    state_perm = state.permute(permuted_list)
    
    operator_list = [a] + [a] * (number_modes - 1)

    q1 = qt.tensor(operator_list) * state_perm
    
    q2 = q1.permute(permuted_list)
    
    
    results = []
    results.append(q2)
    for i in range(number_modes):
        pt = q2.ptrace(i)
        results.append(np.trace(pt * qt.create(N) * qt.destroy(N)))
        
        
    return results